In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

### 저수준의 RDD API 패턴과 고수준 DSL과 데이터 프레임 API를 사용한 예시 비교

In [ ]:
# RDD를 이용한 예제
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
sc = spark.sparkContext
# (name, age) 형태의 튜플로 된 RDD 생성
dataRDD = sc.parallelize([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)])
# 집계와 평균을 위한 람다 표현식, map, reduceByKey transformation
ageRDD = (dataRDD
          .map(lambda x: (x[0], (x[1], 1)))
          .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
          .map(lambda x: (x[0], x[1][0] / x[1][1]))
          )
print(ageRDD.collect())
spark.stop()

In [ ]:
# 고수준 DSL, 데이터프레임 API 사용
from pyspark.sql.functions import avg
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

data_df = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), ("Brooke", 25)], ['name', 'age'])
avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

spark.stop()

### 스키마 정의 방법

In [ ]:
# 프로그래밍 스타일
from pyspark.sql.types import *
schema = StructType([StructField("author", StructType(), False),
                     StructField("title", StringType(), False),
                     StructField("pages", IntegerType(), False)])

In [ ]:
# DDL 사용
schema = "author STRING, title STRING, pages INT"

### 로우 객체 이해하기

In [ ]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter", "LinkedIn"])
blog_row[1]

In [ ]:
# 데이터 프레임으로 만들어서 사용
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

rows = [Row("Matei Zaharia", "CA"),Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

spark.stop()

### 샌프란시스코 데이터 예제

데이터 불러오기

In [ ]:
# schema 정의
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
fire_df = spark.read.csv('sf-fire-calls.csv', header=True)
fire_df.show(5)

트랜젝션과 필터

In [ ]:
from pyspark.sql.functions import *

# CallType이 "Medical Incident"가 아닌 데이터 필터
few_fire_df = (fire_df
               .select("IncidentNumber", "AvailableDtTM", "CallType")
               .where(col("CallType") != "Medical Incident"))
few_fire_df.show(5, truncate=False)

In [ ]:
# CallType의 종류가 몇 가지인지 세어주는 코드
(fire_df.select("CallType")
 .where(col("CallType").isNotNull())
 .agg(countDistinct("CallType").alias("DistinctCallTypes"))
 .show()
 )

In [ ]:
# 모든 행에서 null이 아닌 개별 CallType을 추출
(fire_df
 .select("CallType")
 .where(col("CallType").isNotNull())
 .distinct()
 .show(10, False)
 )

칼럼의 이름 변경 및 추가 삭제

In [ ]:
new_fire_df = fire_df.withColumnRenamed("Delay", "changeDelay")
new_fire_df.select("changeDelay").show(5)

In [ ]:
(new_fire_df
 .select("CallDate", "WatchDate", "AvailableDtTm")
 .show(5, False))

In [ ]:
# 문자열 타입의 날짜 데이터를 date 타입의 데이터로 변환하는 코드
fire_ts_df = (new_fire_df
              .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
              .drop("CallDate")
              .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
              .drop("WatchDate")
              .withColumn("AvailableDtTs", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
              .drop("AvailableDtTm"))

(fire_ts_df
 .select("IncidentDate", "OnWatchDate", "AvailableDtTs")
 .show(5, False))

In [ ]:
# 데이터에 포함된 연도의 종류를 뽑아내는 코드
(fire_ts_df
 .select(year("IncidentDate"))
 .distinct()
 .orderBy(year('IncidentDate'))
 .show())

집계연산

In [12]:
# CallType의 집계
(fire_ts_df
 .select("CallType")
 .where(col("CallType").isNotNull())
 .groupBy("CallType")
 .count()
 .orderBy("count", ascending = False)
 .show(n = 10, truncate = False))

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [1]:
spark.stop()

NameError: name 'spark' is not defined